In [1]:
from google.colab import drive
drive.mount('/gdrive')
import os

os.chdir('/gdrive/MyDrive/')


Mounted at /gdrive


In [2]:
!pip install GPy
!pip install sklearn
!pip install tensorboardX

     |████████████████████████████████| 1.0MB 8.3MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
  Created wheel for GPy: filename=GPy-1.9.9-cp37-cp37m-linux_x86_64.whl size=2626955 sha256=7670000653d98dc5b297d11e634690da7d094e04984823799631bef7b9ee9109
  Stored in directory: /root/.cache/pip/wheels/5d/36/66/2b58860c84c9f2b51615da66bfd6feeddbc4e04d887ff96dfa
  Created wheel for paramz: filename=paramz-0.9.5-cp37-none-any.whl size=102552 sha256=74ec3f5e01270a659671bda02f9d7be339aca3a08f05ededf46f737eea8aab65
  Stored in directory: /root/.cache/pip/wheels/c8/4a/0e/6e0dc85541825f991c431619e25b870d4b812c911214690cf8
Successfully built GPy paramz
     |████████████████████████████████| 122kB 9.8MB/s 


In [3]:
#The 3 following cells aim at simuling the behavior of NN models with a simple model for MNIST
!pip install ray==1.2.0
!pip install -U hyperopt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import torch.nn.functional as F
import math
import numpy as np
import random
import math 
from ray import tune
from hyperopt import hp, fmin, tpe, Trials
from functools import *
from ray.tune.logger import *
import time

EPOCH_SIZE = 32*32*32*32
TEST_SIZE = 256*32*32 #remove 1024

#This is a function that can be used by several NN model
def train(model, optimizer ,func ,train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    #for (data, target) in train_loader:
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
           # print("hehe")
            return
        # We set this just for the example to run quickly.
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = func(output, target)
        loss.backward()
        optimizer.step()
        
#This is a function that can be used by several NN model (it only does accuracy ATM)
def test(model, func, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)

            total += target.size(0)
            correct += (predicted == target).sum().item()


                
    return correct / total

     |████████████████████████████████| 47.5MB 62kB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 1.3MB 49.2MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 204kB 52.0MB/s 
     |████████████████████████████████| 133kB 45.3MB/s 
     |████████████████████████████████| 3.1MB 48.3MB/s 
     |████████████████████████████████| 143kB 58.2MB/s 
     |████████████████████████████████| 296kB 51.1MB/s 
     |████████████████████████████████| 92kB 11.0MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp37-none-any.whl size=12621 sha256=75319235aef5efd6f4b3efe077d8459d360e33bd603438dce91ba54d6b718081
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
Successfully built gpustat
     |████████████████████████████████| 972kB 7.1MB/s 
  Found existing installation: hyperopt 0.1.2
   

In [4]:
# A random mnist from the internet to get a correct model to reason about

class train_mnist():
    def __init__(self,config):
        
        self.config = {
        "sigmoid_func": 1
      ,  "hidden_dim":64
      ,  "n_layer":3    }
        for key, value in config.items():
            self.config[key] = value
        config = self.config
        
        self.i = 0
        
       # mnist_transforms = transforms.Compose(
       #     [transforms.ToTensor(),
       #      transforms.Normalize((0.1307, ), (0.3081, ))])
        mnist_transforms = transforms.ToTensor()

        self.train_loader = DataLoader(
            datasets.FashionMNIST("/gdrive/MyDrive", train=True, download=True , transform=mnist_transforms),
            batch_size=1024,
            shuffle=True)
       # self.test_loader = DataLoader(
       #     datasets.MNIST("/gdrive/MyDrive", train=False, transform=mnist_transforms),
       #     batch_size=64,
       #     shuffle=True)

        
        test_valid_dataset = datasets.FashionMNIST("/gdrive/MyDrive", train=False, transform=mnist_transforms)
        valid_ratio = 0.5  
        nb_test = int((1.0 - valid_ratio) * len(test_valid_dataset))
        nb_valid =  int(valid_ratio * len(test_valid_dataset))
        test_dataset, val_dataset = torch.utils.data.dataset.random_split(test_valid_dataset, [nb_test, nb_valid])
        self.test_loader =  DataLoader(test_dataset,
            batch_size=1024,
            shuffle=True)

        self.val_loader =  DataLoader(val_dataset,
            batch_size=1024,
            shuffle=True)

        sigmoid_func_uniq = nn.Tanh()


        self.model = LeNet(192,64,10,
                    3,
                    config.get("droupout_prob",0.5) ,sigmoid_func_uniq)
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config.get("lr", 0.01), 
                                     betas=((config.get("b1", 0.999),config.get("b2", 0.9999))),
                                     eps=config.get("eps", 1e-08), 
                                     weight_decay=config.get("weight_decay", 0), 
                                     amsgrad=True)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
    
    def adapt(self, config):
        #print(self.optimizer)
        temp = copy.deepcopy(self)
        for key, value in config.items():
            temp.config[key] = value
        config = temp.config

        temp.model.adapt(config.get("droupout_prob", 0.5))
        temp.optimizer = torch.optim.Adam(temp.model.parameters(), lr=config.get("lr", 0.01), 
                                     betas=((config.get("b1", 0.999),config.get("b2", 0.9999))),
                                     eps=config.get("eps", 1e-08), 
                                     weight_decay=config.get("weight_decay", 0), 
                                     amsgrad=True)
        return temp
    
    
# All NN models should have a function train1 and test1 that calls the common train and test defined above.
# train1 and test1 is then used in the scheduler
    def train1(self):
        print("iteration: " + str(self.i) )
        self.i+=1
        train(self.model, self.optimizer, F.nll_loss, self.train_loader)

    def val1(self):
        return test(self.model, F.nll_loss, self.val_loader)
        
    def test1(self):
        return test(self.model, F.nll_loss, self.test_loader)

    def step(self):
        train1()
        return val1()

# __INCEPTION_SCORE_begin__
class LeNet(nn.Module):
    """
    LeNet for MNist classification, used for inception_score
    """

    def __init__(self,input_dim, hidden_dim, output_dim, n_layers,
            drop_prob, sigmoid ):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(drop_prob)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def adapt(self,drop_prob):
        self.conv2_drop = nn.Dropout2d(drop_prob)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)



# Convolution Neural network using Pytorch 
class ConvNet(nn.Module):
    def __init__(self,input_dim, hidden_dim, output_dim, n_layers,
                 drop_prob, sigmoid ):
        super(ConvNet, self).__init__()
        
        self.sigmoid = sigmoid
        self.i_d = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)

        self.fc = nn.Linear(input_dim, output_dim)
        self.first= nn.Linear(input_dim, hidden_dim)
        self.hidden = [nn.Linear(hidden_dim,hidden_dim) for _ in range(self.n_layers)]
        self.drop_out = nn.Dropout(drop_prob)

        self.last = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.sigmoid(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, self.i_d)
        x=self.first(x)
        x=self.drop_out(x)
        for i in range(self.n_layers):
            x=self.hidden[i](x)
            x=self.drop_out(x)
        x = self.last(x)
        return F.log_softmax(x, dim=1)


In [5]:
class Parent():
    """Parent Class that handles the passage of Network Configurations from one step to the
    following
    """
    def __init__(self, point_hyperspace, configuration, model, loss):
        self.point_hyperspace = point_hyperspace
        self.configuration_list = [configuration]
        self.loss_list = [np.array(loss)]
        self.model = model
        self.is_replicated = False

    def update(self, configuration, loss, model):
        self.is_replicated = False
        self.configuration_list.append(configuration)
        self.loss_list=np.append(self.loss_list,loss)
        self.model = model

    def replication(self, n_children):
        self.is_replicated = True
      #  self.configuration_list.append(self.configuration_list[-1])
      #  self.loss_list=np.append(self.loss_list,self.loss_list[-1])
    #    replication_trials(self.point_hyperspace.trials, n_children)

    def get_last_conf(self):
        return self.configuration_list[-1]

    def get_point_hyperspace(self):
        return self.point_hyperspace

    def get_model(self):
        return self.model

    def get_loss(self):
        return self.loss_list

0.3674


0.6875


In [10]:
import copy


class Scheduler():
    def __init__(self, model, num_iteration, num_config,
                 oracle):
        #Oracle manages the Bayesian optimization
        self.oracle = oracle
        self.iteration = num_iteration
        self.num_config = num_config 
        self.sqrt_config =   math.floor(math.sqrt(num_config/4)) # math.ceil(num_config/5) #
        self.n_parents = self.sqrt_config
        #self.h is for the m "h" used at every loop, h is a configuration from the search space
        self.h = np.repeat({},num_config) 
        
        #self.out is for storing the result of the algorithm, ie all "h" from all iterations
        #from all sqrt(m) best models per iterations.
        self.out = np.zeros((num_iteration,self.sqrt_config))
        
        #self.hyperspaces is for storing the sqrt(m) hyperspaces used by the algorithm
        self.hyperspaces = np.zeros(self.sqrt_config)
        
        self.plot = np.zeros(num_iteration)

        
        #self.model is the m model that will explore new hyperspace points at every iterations
        self.models = np.repeat(model,num_config)
        
        #self.parents is the sqrt(m) best model from last iteration
        self.parents = np.repeat(model,self.sqrt_config)

        #self.losses remembers the performances of all m models at one iteration to decide which ones are the sqrt(m) best from self.models.
        self.losses = np.zeros(num_config)
        
        self.k = [0] # c'est pour avoir un pointeur sur k, c'est pas plus que O(sqrt)-paralélisable  pour le moment du coup.
    
    def initialisation(self):
        num_config = self.num_config
        extended_Hyperspace = Trials()
        fmin_objective = partial(test_function, models=self.models,h=self.h,losses=self.losses,parent_model=self.models, k_f = self.k,iteration = 0)
        self.oracle.compute_batch(extended_Hyperspace ,num_config , 0 ,fmin_objective)
            
        indexes = np.argsort(self.losses)     
        self.out[0] = (self.losses[indexes])[0:self.sqrt_config]
        self.hyperspaces = np.repeat(extended_Hyperspace,self.sqrt_config)    
        self.parents = np.array([Parent(copy.deepcopy(extended_Hyperspace),(self.h[indexes])[i], (self.models[indexes])[i],(self.losses[indexes])[i])  
                                 for i in range(self.sqrt_config) ])         
        self.plot[0] = self.losses[indexes][0]
        
    def loop(self):
        sqrt_config = self.sqrt_config
        iteration = self.iteration
        for i in range(1,iteration):
            
            self.k[0] = 0
            
            start_time = time.time()
            for j in range(sqrt_config):
                parent = self.parents[j]
                point_extended_hyperspace = parent.get_point_hyperspace()
                print("\n loss of parent " + str(parent.get_loss()[-1]) )
                print("\n loss " + str(parent.get_loss()))
                
                fmin_objective = partial(test_function, models=self.models,h=self.h,losses=self.losses,parent_model=parent.get_model(), k_f = self.k,iteration = len(parent.get_loss()))

                    
                if not parent.is_replicated:
                    print('not replicated')
                    self.oracle.repeat_good(
                        point_extended_hyperspace,
                        len(parent.get_loss()),
                        fmin_objective,
                        parent.configuration_list[-1]
                    )

                    # computes the new batch for each one of the parents for every iteration
                    self.oracle.compute_batch(
                        point_extended_hyperspace,
                        int(self.num_config/self.n_parents) - 1,
                        len(parent.get_loss()),
                        fmin_objective
                    )
                else:

                    print('replicated')
                    self.oracle.compute_batch(
                        point_extended_hyperspace,
                        int(self.num_config/self.n_parents),
                        len(parent.get_loss()),
                        fmin_objective
                    )
                    

            #self.oracle.Repeat_good(extended_Hyperspace ,i ,fmin_objective,parent.configuration_list[-1])
             #   self.oracle.compute_Batch(extended_Hyperspace ,int(self.num_config/sqrt_config) -1 , i ,fmin_objective)
           
            print("totalt time: " +  str(time.time() - start_time))


            combined_losses = np.concatenate(
                        (
                            self.losses,
                            
                                [self.parents[i].get_loss()[-1].item() for i in range(self.n_parents)]
                            
                        ),
                        0
                    )
            ixs_parents = np.argsort(combined_losses)
            parent_idx = ixs_parents[:self.n_parents]
            print(combined_losses)
            print(parent_idx)
            # ??? why saving it in a numpt array ?
            # It is creating the new Parent `array`
            temp_parents = [''] * self.n_parents

            for j, x in enumerate(parent_idx):
                # ??? why converting it to integer ?
                x = int(x)
                if x >= self.num_config:
                    temp_parents[j] = copy.deepcopy(self.parents[x - self.num_config])
                    temp_parents[j].replication(self.n_parents)
                else:
                    temp_parents[j] = copy.deepcopy(self.parents[math.floor(x/self.num_config * self.n_parents)])
                    temp_parents[j].update(self.h[x], self.losses[x], self.models[x])
                   # temp_parents[j].point_hyperspace = Trials()

            self.parents = temp_parents


class FSVNLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "FSNV_FMNIST_timeTPEcis1-4v1.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()

            #if not self._continuing:
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   

config= {
     "lr": hp.loguniform("lr",-5*2.3,-1*2)
    , "droupout_prob": hp.uniform("droupout_prob",0,1)
          ,   "weight_decay": hp.loguniform("weight_decay",-5*2.3,-1*2),
    "b1" : 1-hp.loguniform("b1",-4*2.3, -1*2.3),
    "b2" : 1-hp.loguniform("b2",-5*2.3, -2*2.3)
}
fsvnlogger = FSVNLogger(config,"")

    
CONFIGURATION = 4
ITERATIONS = 20

model = train_mnist
oracle = Oracle(config)


scheduler = Scheduler(
  model,
  ITERATIONS,
  CONFIGURATION,
  oracle) 

start_time = time.time()
#scheduler.initialisation()     
#scheduler.loop()     
print("totalt time: " +  str(time.time() - start_time))


totalt time: 5.030632019042969e-05


In [ ]:
for _ in range(6):


  scheduler = Scheduler(
    model,
    ITERATIONS,
    CONFIGURATION,
    oracle) 

  start_time = time.time()
  scheduler.initialisation()     
  scheduler.loop()     
  print("totalt time: " +  str(time.time() - start_time))



iteration: 0
accuracy, 0.4294

iteration: 0
accuracy, 0.3006

iteration: 0
accuracy, 0.1648

iteration: 0
accuracy, 0.1032

100%|██████████| 4/4 [00:31<00:00,  7.86s/trial, best loss: -0.4294]

 loss of parent -0.4294

 loss [array(-0.4294)]
not replicated
iteration: 1
accuracy, 0.6336

100%|██████████| 5/5 [00:04<00:00,  1.16trial/s, best loss: -0.6336]
iteration: 1
accuracy, 0.3812

iteration: 1
accuracy, 0.3562

iteration: 1
accuracy, 0.4396

100%|██████████| 8/8 [00:13<00:00,  1.63s/trial, best loss: -0.6336]
totalt time: 17.36221742630005
[-0.6336 -0.3812 -0.3562 -0.4396 -0.4294]
[0]

 loss of parent -0.6336

 loss [-0.4294 -0.6336]
not replicated
iteration: 2
accuracy, 0.673

100%|██████████| 9/9 [00:04<00:00,  2.10trial/s, best loss: -0.673]
iteration: 2
accuracy, 0.3634

iteration: 2
accuracy, 0.6378

iteration: 2
accuracy, 0.718

100%|██████████| 12/12 [00:13<00:00,  1.09s/trial, best loss: -0.718]
totalt time: 17.45829153060913
[-0.673  -0.3634 -0.6378 -0.718  -0.6336]
[3]

 

In [ ]:
for _ in range(10):
  oracle = BayesOpt(config)
  start_time = time.time()
  oracle.compute_Once(function)
  print("totalt time: " +  str(time.time() - start_time))

{'b1': 0.999489475480056, 'b2': 0.9996101399389558, 'droupout_prob': 0.0016392466537953343, 'lr': 0.0011726065238878184, 'weight_decay': 0.036559821439183605}
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
{'b1': 0.9997753063344094, 'b2': 0.9995832912492338, 'droupout_prob': 0.12525174131699018, 'lr': 0.0007467333184929997, 'weight_decay': 0.12954286364071502}
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
{'b1': 0.9738752663848531, 'b2': 0.9999889803548367, 'droupout_prob': 0.004633335271537163, 'lr': 0.0749308436789229, 'weight_decay': 0.00013106

[]

In [7]:

# Oracle (Paul) TODO
path = "/gdrive/MyDrive/"
class Oracle():
    def __init__(self, searchspace ):
        #self.hyperspace is the original (input) searchspace
        self.searchspace = searchspace

    def repeat_good(self,trials, iteration,function,configuration): #add space
        space = copy.deepcopy(configuration)
        for k,v in configuration.items():
            space[k] =  hp.uniform(k,-1e-10+v,v + 1e-10) 

        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+1, trials=trials)
        
    def compute_once(self,trials, iteration,function): #add space

        space = copy.deepcopy(self.searchspace)
        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+1, trials=trials)
        
        
    def compute_batch(self,trials, nb_eval, iteration,function): #add space

        space = copy.deepcopy(self.searchspace)
        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
            
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+nb_eval, trials=trials)
    


In [8]:
def test_function(x,models,h,losses, parent_model,k_f,iteration):
    if (isinstance(k_f,list)):
            k=k_f[0]
            Islist = True 
    else:
            k = k_f
            Islist = False
        
    if iteration == 0:
        models[k] = parent_model[k](x)
    else:      
        models[k] = parent_model.adapt(x)
    if(Islist):
        k_f[0] += 1
    
    #for key, value in x.items():
    #        print(key + " "+str(x[key]))

    h[k] = x
    #start_time = time.time()
    models[k].train1()
    loss = models[k].test1()
    test = models[k].val1()
    #print("--- %s seconds ---" % (time.time() - start_time))

    temp = dict(x)
    temp.update({'loss' : loss})
    temp.update({'test' : test})
    fsvnlogger.on_result(temp)

    losses[k] = -loss
    print("accuracy, " + str(loss) + "\n")
    return -loss

In [8]:
#Normal Random Opt
evals = 4
crypt = np.zeros(evals)
def function(x):
    print(x)
    model = train_mnist(x)
    for _ in range(20):
        model.train1()
        loss = model.test1()
        test = model.val1()
        temp = dict(x)
        temp.update({'loss' : loss})
        temp.update({'test' : test})

        temp.update({'iteration' :  model.i})
        hyperlogger.on_result(temp)
    
  #  loss = model.test1()
    return -loss

In [9]:
class RandomOpt():
    def __init__(self, searchspace ):
        self.searchspace = searchspace

    def compute_Once(self,function): 
        fmin(function, self.searchspace, algo=partial(tpe.rand.suggest), max_evals=evals, trials=Trials())
    
config= {
     "lr": hp.uniform("lr",0,.1),
     "droupout_prob": hp.uniform("droupout_prob",0,1),
     "weight_decay": hp.uniform("weight_decay",0,.1),
    "b1" : hp.uniform("b1",0.9, 1),
    "b2" : hp.uniform("b2",0.99, 1)
    
}
oracle = RandomOpt(config)

In [10]:
class RandomLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "trash.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()

            #if not self._continuing:
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   
hyperlogger = RandomLogger(config,"")

start_time = time.time()
oracle.compute_Once(function)
print("totalt time: " +  str(time.time() - start_time))


{'b1': 0.9745969685466439, 'b2': 0.9989423610987771, 'droupout_prob': 0.16604364131613214, 'lr': 0.052512761326331515, 'weight_decay': 0.05816915838646053}
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
{'b1': 0.9121331281586186, 'b2': 0.9982841479559941, 'droupout_prob': 0.7591039754613024, 'lr': 0.08618807382488797, 'weight_decay': 0.02819455604568967}
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
{'b1': 0.9203038527215194, 'b2': 0.9935756271225604, 'droupout_prob': 0.3093385735919343, 'lr': 0.00047119234646567646, 'weight_decay': 0.008625820776

In [ ]:
class BayesOpt():
    def __init__(self, searchspace ):
        self.searchspace = searchspace

    def compute_Once(self,function): 
        fmin(function, self.searchspace, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=evals, trials=Trials())
    
config= {
     "lr": hp.loguniform("lr",-5*2.3,-1*2)
    , "droupout_prob": hp.uniform("droupout_prob",0,1)
          ,   "weight_decay": hp.loguniform("weight_decay",-5*2.3,-1*2),
    "b1" : 1-hp.loguniform("b1",-4*2.3, -1*2.3),
    "b2" : 1-hp.loguniform("b2",-5*2.3, -2*2.3)
}
oracle = BayesOpt(config)

In [ ]:
class HyperLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "hyperoptFMNIST.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()

            #if not self._continuing:
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   
hyperlogger = HyperLogger(config,"")

  


In [11]:
# A random mnist from the internet to get a correct model to reason about
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
#from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import json
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.schedulers import AsyncHyperBandScheduler
import torch
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn

import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler 
from ray.tune.schedulers import MedianStoppingRule

from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
#from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import json
import os
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
#from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.schedulers import AsyncHyperBandScheduler
import torch
#import adabelief_pytorch
global_checkpoint_period=np.inf
from ray.tune.schedulers.pb2 import PB2

class train_mnist_pb2(tune.Trainable):
    def setup(self, config):
        self.config = {
        "sigmoid_func": 1
      ,  "hidden_dim":43
      ,  "n_layer":2    }
        for key, value in config.items():
            self.config[key] = value
        config = self.config
        
        self.i = 0
        
        #mnist_transforms = transforms.Compose(
        #    [transforms.ToTensor(),
        #     transforms.Normalize((0.1307, ), (0.3081, ))])
        mnist_transforms = transforms.ToTensor()

        self.train_loader = DataLoader(
            datasets.FashionMNIST("/gdrive/MyDrive", train=True, download=True, transform=mnist_transforms),
            batch_size=1024,
            shuffle=True)
       # self.test_loader = DataLoader(
       #     datasets.MNIST("~/data", train=False, transform=mnist_transforms),
       #     batch_size=64,
       #     shuffle=True)

        
        test_valid_dataset = datasets.FashionMNIST("/gdrive/MyDrive", train=False, transform=mnist_transforms)
        valid_ratio = 0.5  
        nb_test = int((1.0 - valid_ratio) * len(test_valid_dataset))
        nb_valid =  int(valid_ratio * len(test_valid_dataset))
        test_dataset, val_dataset = torch.utils.data.dataset.random_split(test_valid_dataset, [nb_test, nb_valid])
        self.test_loader =  DataLoader(test_dataset,
            batch_size=1024,
            shuffle=True)

        self.val_loader =  DataLoader(val_dataset,
            batch_size=1024,
            shuffle=True)

        sigmoid_func_uniq = nn.Tanh()


        self.model = LeNet(192,64,10,
                    3,
                    config.get("droupout_prob",0.5) ,sigmoid_func_uniq)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        if config.get("b1", 0.999)>=1:
          temp = 1 - 1e-10
        else:
          temp = 1-config.get("b1", 0.999)
                
        if config.get("b2", 0.999)>=1:
          temp1 = 1 - 1e-10
        else:
          temp1 = 1-config.get("b2", 0.999)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config.get("lr", 0.01), 
                                     betas=((temp,temp1)),
                                     eps=config.get("eps", 1e-08), 
                                     weight_decay=config.get("weight_decay", 0), 
                                     amsgrad=True)

    

    
    def reset_config(self, config):
        if "lr" in config:
            for param_group in self.optimizer.param_groups:
                param_group["lr"] = config.get("lr", 0.01)
        if "b1" in config:
            for param_group in self.optimizer.param_groups:
                param_group["betas"] = (
                    (1-1e-10 if config.get("b1", 0.999)>=1 else 1-config.get("b1", 0.999),(1-1e-10 if config.get("b2", 0.999)>=1 else 1-config.get("b2", 0.999))))
        if "wd" in config:
            for param_group in self.optimizer.param_groups:
                param_group["weight_decay"] = weight_decay
        
        self.model.adapt(config.get("droupout_prob", 0.5))
        self.config = config
        return True
    
# All NN models should have a function train1 and test1 that calls the common train and test defined above.
# train1 and test1 is then used in the scheduler
    def train1(self):
        print("iteration: " + str(self.i) )
        self.i+=1
        train(self.model, self.optimizer, F.nll_loss, self.train_loader)

    def val1(self):
        return test(self.model, F.nll_loss, self.val_loader)
        
    def test1(self):
        return test(self.model, F.nll_loss, self.test_loader)

    def step(self):
        self.train1()
        return {'loss' : self.test1(), 'test' : self.val1()}
    
    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        torch.save({
            "model": self.model.state_dict(),
            "optim": self.optimizer.state_dict(),

        }, path)

        return checkpoint_dir

    def load_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint["model"])
        self.optimizer.load_state_dict(checkpoint["optim"])
epsilon = 1e-10



algo = HyperOptSearch(metric="loss",
    mode="max")
algo = ConcurrencyLimiter(algo, max_concurrent=25)
scheduler = PB2(
    time_attr="training_iteration",
    perturbation_interval=1,
    hyperparam_bounds={
        # distribution for resampling
     "lr": [1e-5,1e-1] #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":[1e-5,1e-1]#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": [1e-4, 1e-1]#,1e-4), #*10 et 0
 ,    "b2": [1e-5, 1e-2] #,1e-4), #*10 et 0
 ,    "droupout_prob": [0+epsilon, 1-epsilon]#,1e-4), #*10 et 0
    }) 

imageSize = 32


class TestLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "trash.csv") #aller jusqu'a 9
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   

cst=imageSize
total=cst*cst*3
def trial_name_id(trial):
    return f"{trial.trainable_name}_{trial.trial_id}"
from ray import tune


for _ in range(1):
  ray.shutdown()
  ray.init()
  start_time = time.time()
  analysis = tune.run(
      
  train_mnist_pb2,
  scheduler=scheduler,
  reuse_actors=False,
  search_alg=algo,
  verbose=2,
  checkpoint_at_end=True,
  num_samples=4,
  # export_formats=[ExportFormat.MODEL],
config= {
     "lr": tune.loguniform(1e-5,1e-1)
    , "droupout_prob": tune.uniform(0,1)
          ,   "weight_decay": tune.loguniform(1e-5,1e-1),
    "b1" : tune.loguniform(1e-4, 1e-1),
    "b2" : tune.loguniform(1e-5, 1e-2)
},      stop={
          "training_iteration": 20,
      },        metric="loss",
      mode="max"
,resources_per_trial={'cpu':2 ,'gpu': 1}
              ,     loggers=[TestLogger])
  print("time "+ str(time.time()- start_time))

2021-04-09 14:54:43,419	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265


(pid=1546) iteration: 0
(pid=1546) iteration: 0
(pid=1546) iteration: 0
Trial train_mnist_pb2_86d17cf2 reported loss=0.64 with parameters={'lr': 0.0009061588195672375, 'droupout_prob': 0.2692041796099871, 'weight_decay': 1.283284672648546e-05, 'b1': 0.00016172356624490303, 'b2': 0.00013156179531585805}.


(pid=1545) iteration: 0
(pid=1545) iteration: 0
(pid=1545) iteration: 0


2021-04-09 14:55:09,999	INFO pbt.py:482 -- [pbt]: no checkpoint for trial. Skip exploit for Trial train_mnist_pb2_86e06280
2021-04-09 14:55:10,107	WARNING trial_runner.py:420 -- Trial Runner checkpointing failed: Checkpoint must not be in-memory.


Trial train_mnist_pb2_86e06280 reported loss=0.61 with parameters={'lr': 0.007371981970676272, 'droupout_prob': 0.9381541388122954, 'weight_decay': 0.0012629424710414985, 'b1': 0.03961254656016891, 'b2': 0.004477282354955519}.


...

(pid=3709) 2021-04-09 15:14:48,697	INFO trainable.py:372 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/train_mnist_pb2_2021-04-09_14-54-46/train_mnist_pb2_94b046aa_4_b1=0.00082823,b2=0.00090637,droupout_prob=0.82585,lr=0.0045291,weight_decay=1.2662e-05_2021-04-09_14-55-22/tmpk6wgto1urestore_from_object/./
(pid=3709) 2021-04-09 15:14:48,698	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 19, '_timesteps_total': None, '_time_total': 153.4873607158661, '_episodes_total': None}
(pid=3709) 2021-04-09 15:14:48,697	INFO trainable.py:372 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/train_mnist_pb2_2021-04-09_14-54-46/train_mnist_pb2_94b046aa_4_b1=0.00082823,b2=0.00090637,droupout_prob=0.82585,lr=0.0045291,weight_decay=1.2662e-05_2021-04-09_14-55-22/tmpk6wgto1urestore_from_object/./
(pid=3709) 2021-04-09 15:14:48,698	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 19, '_timesteps_total': None, '_time_total': 153.487360

Trial train_mnist_pb2_94b046aa reported loss=0.79 with parameters={'lr': 1e-05, 'droupout_prob': 1e-10, 'weight_decay': 1e-05, 'b1': 1e-04, 'b2': 0.01}. This trial completed.


Trial name,status,loc,b1,b2,droupout_prob,lr,weight_decay,iter,total time (s),loss,test
train_mnist_pb2_86d17cf2,TERMINATED,,0.0443773,0.00535319,1e-10,0.0466309,0.053197,20,161.405,0.7838,0.7842
train_mnist_pb2_86e06280,TERMINATED,,0.1,0.00553075,1e-10,0.1,0.0452919,20,161.62,0.787,0.7878
train_mnist_pb2_8de91efa,TERMINATED,,0.0565353,0.01,1e-10,0.0670965,0.1,20,161.629,0.7682,0.7974
train_mnist_pb2_94b046aa,TERMINATED,,0.0001,0.01,1e-10,1e-05,1e-05,20,161.835,0.7882,0.7816


2021-04-09 15:14:57,137	INFO tune.py:450 -- Total run time: 1210.33 seconds (1210.23 seconds for the tuning loop).


time 1210.3524842262268


In [ ]:
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import *
class TestLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "BOHB_FMNISTvalid.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()

config= {
     "lr": tune.loguniform(1e-5,1e-1)
    , "droupout_prob": tune.uniform(0,1)
          ,   "weight_decay": tune.loguniform(1e-5,1e-1),
    "b1" : tune.loguniform(1e-4, 1e-1),
    "b2" : tune.loguniform(1e-5, 1e-2)
}

for i in range(10):
  class TestLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "BOHB_FMNISTvalid"+str(i)+".csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
    ray.shutdown()
    ray.init()
    start_time = time.time()
    algo = TuneBOHB(metric="loss", mode="max")
    bohb = HyperBandForBOHB(
        time_attr="training_iteration",
        metric="loss",
        mode="max",
        max_t=20)
    analysis = tune.run(train_mnist_pb2, config=config, scheduler=bohb, search_alg=algo,num_samples=4,loggers=[TestLogger],resources_per_trial={'cpu':2 ,'gpu': 1})

    print("time "+ str(time.time()- start_time))

2021-04-08 21:49:03,127	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265


(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
(pid=32701) iteration: 0
Trial train_mnist_pb2_3d82d666 reported loss=0.1166,test=0.1026 with parameters={'lr': 4.6267943965665504e-05, 'droupout_prob': 0.21113086137252124, 'weight_decay': 0.01543712450627116, 'b1': 0.0006171820502638937, 'b2': 5.4255211048484266e-05}.


(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32701) iteration: 1
(pid=32702) iteration: 0
(pid=32702) iteration: 0
(pid=32702) iteration: 0
(pid=32702) iteration: 0
(pid=32702) iteration: 0
(pid=32702) iteration: 0
(pid=32702) iteration: 0
(pid=32702) iteration: 0
(pid=32702) iteration: 0
(pid=32702) iteration: 0
(pid=32702) iteration: 0
(pid=32702) iteration: 0


(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1
(pid=32702) iteration: 1


2021-04-08 21:49:33,423	WARNING trial_runner.py:420 -- Trial Runner checkpointing failed: Checkpoint must not be in-memory.


...

2021-04-08 22:27:48,966	WARNING trial_runner.py:420 -- Trial Runner checkpointing failed: Checkpoint must not be in-memory.
2021-04-08 22:27:48,976	WARNING tune.py:429 -- Trial Runner checkpointing failed: Checkpoint must not be in-memory.


Trial train_mnist_pb2_1d04183c reported loss=0.7784,test=0.7706 with parameters={'lr': 0.0002806516532161078, 'droupout_prob': 0.2746398003529301, 'weight_decay': 0.0020716815018807172, 'b1': 0.008574107954303242, 'b2': 0.0013950214804238923}.
Trial train_mnist_pb2_2506016c reported loss=0.7986,test=0.8018 with parameters={'lr': 0.020201325161436905, 'droupout_prob': 0.2676566382153672, 'weight_decay': 0.016141425807639272, 'b1': 0.0006441474533927233, 'b2': 0.009459437415424442}. This trial completed.


Trial name,status,loc,b1,b2,droupout_prob,lr,weight_decay,iter,total time (s),loss,test
train_mnist_pb2_1d04183c,TERMINATED,,0.00857411,0.00139502,0.27464,0.000280652,0.00207168,14,63.118,0.7784,0.7706
train_mnist_pb2_1d1242fe,TERMINATED,,0.000210349,9.44403e-05,0.556937,1.48575e-05,0.00466355,6,27.0677,0.2146,0.2268
train_mnist_pb2_2506016c,TERMINATED,,0.000644147,0.00945944,0.267657,0.0202013,0.0161414,14,64.378,0.7986,0.8018
train_mnist_pb2_2cf61560,TERMINATED,,0.0659238,0.000799985,0.266513,0.00156905,0.0499471,6,27.3486,0.711,0.6972


2021-04-08 22:27:48,999	INFO tune.py:450 -- Total run time: 230.23 seconds (230.17 seconds for the tuning loop).


time 230.25683045387268


In [ ]:
!pip install ConfigSpace
!pip install hpbandster


     |████████████████████████████████| 952kB 7.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for ConfigSpace: filename=ConfigSpace-0.4.18-cp37-cp37m-linux_x86_64.whl size=2879782 sha256=25b20bedea4b08819cc1abefb5bd949d0e4916d6c282df704368ddadcd0d18be
  Stored in directory: /root/.cache/pip/wheels/39/ea/40/d93931850f700427db0a84180829c709d30484c9475040c7bd
Successfully built ConfigSpace
     |████████████████████████████████| 61kB 4.5MB/s 
     |████████████████████████████████| 92kB 5.3MB/s 
  Created wheel for hpbandster: filename=hpbandster-0.7.4-cp37-none-any.whl size=80008 sha256=c1f68d2e092e38860cfcf17104162761b7d3f824111cc5ccf88dea803eb2502d
  Stored in directory: /root/.cache/pip/wheels/9d/57/62/6b00c8011bac96e0c404adc5be4e16964ba4544614240b4e23
  Created wheel for netifaces: filename=netifaces-0.10.9-cp37-cp37m-linux_x86_64.whl size=37413 sha256=35f096ae7bcabb39e71d6552797ee4

In [7]:


import argparse

import ray
from ray import tune
from ray.tune.schedulers.pb2 import PB2
from ray.tune.schedulers import PopulationBasedTraining

from ray.tune.examples.pbt_function import pbt_function


for i in range(9,11):
    class TestLogger(tune.logger.Logger):
        def _init(self):
            progress_file = os.path.join(path, "PBT_FMNIST"+str(i)+".csv")
            self._continuing = os.path.exists(progress_file)
            self._file = open(progress_file, "a")
            self._csv_out = None
        def on_result(self, result):
            tmp = result.copy()
            result = flatten_dict(tmp, delimiter="/")
            if self._csv_out is None:
                self._csv_out = csv.DictWriter(self._file, result.keys())
                self._csv_out.writeheader()
            self._csv_out.writerow(
                {k: v
                 for k, v in result.items() if k in self._csv_out.fieldnames})
            self._file.flush()

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    args, _ = parser.parse_known_args()
    ray.shutdown()
    ray.init()  # force pausing to happen for test

    epsilon = 1e-10
    from ray.tune.suggest.hyperopt import HyperOptSearch
    from ray.tune.suggest import ConcurrencyLimiter

    algo = HyperOptSearch(metric="loss",
        mode="max")
    algo = ConcurrencyLimiter(algo, max_concurrent=4)

    pbt = PopulationBasedTraining(
        perturbation_interval=1,
            time_attr="training_iteration",

        hyperparam_mutations={
            # hyperparameter bounds.
     "lr": [1e-5,1e-1] #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":[1e-5,1e-1]#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": [1e-4, 1e-1]#,1e-4), #*10 et 0
 ,    "b2": [1e-5, 1e-2] #,1e-4), #*10 et 0
 ,    "droupout_prob": [0+epsilon, 1-epsilon]#,1e-4), #*10 et 0
        })

    analysis = tune.run(
        train_mnist_pb2,
  checkpoint_at_end=True,
        scheduler=pbt,
        metric="loss",
        mode="max",
        search_alg = algo,
        verbose=0,
        stop={
            "training_iteration": 20,
        },
        num_samples=4,
  reuse_actors=True,
loggers=[TestLogger],
        
config= {
     "lr": tune.loguniform(1e-5,1e-1)
    , "droupout_prob": tune.uniform(0,1)
          ,   "weight_decay": tune.loguniform(1e-5,1e-1),
    "b1" : tune.loguniform(1e-4, 1e-1),
    "b2" : tune.loguniform(1e-5, 1e-2)
}
                      ,resources_per_trial={'cpu':2 ,'gpu': 1})

    print("Best hyperparameters found were: ", analysis.best_config)

2021-04-09 14:17:20,375	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
(pid=471) 2021-04-09 14:17:38,474	INFO trainable.py:103 -- Trainable.setup took 13.067 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


(pid=471) iteration: 0
(pid=471) iteration: 1
(pid=471) iteration: 2


2021-04-09 14:18:04,147	WARNING trial_runner.py:420 -- Trial Runner checkpointing failed: Checkpoint must not be in-memory.


(pid=471) iteration: 3


(pid=471) 2021-04-09 14:18:12,368	INFO trainable.py:372 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/train_mnist_pb2_2021-04-09_14-17-24/train_mnist_pb2_4e2d4bd8_1_b1=0.00050733,b2=1.535e-05,droupout_prob=0.43173,lr=1.3164e-05,weight_decay=0.00032691_2021-04-09_14-17-24/tmp5os6cipgrestore_from_object/./
(pid=471) 2021-04-09 14:18:12,368	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 1, '_timesteps_total': None, '_time_total': 8.754124879837036, '_episodes_total': None}


...

(pid=931) iteration: 85
(pid=931) iteration: 85


(pid=931) 2021-04-09 14:40:38,527	INFO trainable.py:372 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/train_mnist_pb2_2021-04-09_14-29-03/train_mnist_pb2_f5c57e46_3_b1=0.0022484,b2=0.0024026,droupout_prob=0.1755,lr=0.00013552,weight_decay=0.055338_2021-04-09_14-29-23/tmpgeuk4p1nrestore_from_object/./
(pid=931) 2021-04-09 14:40:38,527	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 19, '_timesteps_total': None, '_time_total': 149.83163571357727, '_episodes_total': None}
(pid=931) 2021-04-09 14:40:38,527	INFO trainable.py:372 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/train_mnist_pb2_2021-04-09_14-29-03/train_mnist_pb2_f5c57e46_3_b1=0.0022484,b2=0.0024026,droupout_prob=0.1755,lr=0.00013552,weight_decay=0.055338_2021-04-09_14-29-23/tmpgeuk4p1nrestore_from_object/./
(pid=931) 2021-04-09 14:40:38,527	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 19, '_timesteps_total': None, '_time_total': 149.83163571357727, '

(pid=931) iteration: 86
(pid=931) iteration: 86


2021-04-09 14:40:46,293	WARNING trial_runner.py:420 -- Trial Runner checkpointing failed: Checkpoint must not be in-memory.
(pid=931) 2021-04-09 14:40:46,318	INFO trainable.py:372 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/train_mnist_pb2_2021-04-09_14-29-03/train_mnist_pb2_faa3fe10_4_b1=0.00025604,b2=7.282e-05,droupout_prob=0.87538,lr=0.00064233,weight_decay=0.0036722_2021-04-09_14-29-31/tmpg_51fxuqrestore_from_object/./
(pid=931) 2021-04-09 14:40:46,318	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 19, '_timesteps_total': None, '_time_total': 150.174968957901, '_episodes_total': None}
(pid=931) 2021-04-09 14:40:46,318	INFO trainable.py:372 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/train_mnist_pb2_2021-04-09_14-29-03/train_mnist_pb2_faa3fe10_4_b1=0.00025604,b2=7.282e-05,droupout_prob=0.87538,lr=0.00064233,weight_decay=0.0036722_2021-04-09_14-29-31/tmpg_51fxuqrestore_from_object/./
(pid=931) 2021-04-09 14:40:46,318	INFO trai

(pid=931) iteration: 87
(pid=931) iteration: 87
Best hyperparameters found were:  {'lr': 0.1, 'droupout_prob': 1e-10, 'weight_decay': 1e-05, 'b1': 0.0001, 'b2': 1e-05}


In [ ]:
from ray.tune.schedulers.pb2_utils import normalize, optimize_acq, \
            select_length, UCB, standardize, TV_SquaredExp

In [ ]:


import argparse

import ray
from ray import tune
from ray.tune.schedulers.pb2 import PB2
from ray.tune.schedulers import PopulationBasedTraining

from ray.tune.examples.pbt_function import pbt_function


for i in range(2,3):
    class TestLogger(tune.logger.Logger):
        def _init(self):
            progress_file = os.path.join(path, "PBT_FMNIST"+str(i)+".csv")
            self._continuing = os.path.exists(progress_file)
            self._file = open(progress_file, "a")
            self._csv_out = None
        def on_result(self, result):
            tmp = result.copy()
            result = flatten_dict(tmp, delimiter="/")
            if self._csv_out is None:
                self._csv_out = csv.DictWriter(self._file, result.keys())
                self._csv_out.writeheader()
            self._csv_out.writerow(
                {k: v
                 for k, v in result.items() if k in self._csv_out.fieldnames})
            self._file.flush()

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    args, _ = parser.parse_known_args()
    ray.shutdown()
    ray.init()  # force pausing to happen for test

    epsilon = 1e-10
    from ray.tune.suggest.hyperopt import HyperOptSearch
    from ray.tune.suggest import ConcurrencyLimiter

    algo = HyperOptSearch(metric="loss",
        mode="max")
    algo = ConcurrencyLimiter(algo, max_concurrent=4)

    pbt = PopulationBasedTraining(
        perturbation_interval=1,
            time_attr="training_iteration",

        hyperparam_mutations={
            # hyperparameter bounds.
     "lr": [1e-5,1e-1] #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":[1e-5,1e-1]#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": [1e-4, 1e-1]#,1e-4), #*10 et 0
 ,    "b2": [1e-5, 1e-2] #,1e-4), #*10 et 0
 ,    "droupout_prob": [0+epsilon, 1-epsilon]#,1e-4), #*10 et 0
        })

    analysis = tune.run(
        train_mnist_pb2,
  checkpoint_at_end=True,
        scheduler=pbt,
        metric="loss",
        mode="max",
        search_alg = algo,
        verbose=0,
        stop={
            "training_iteration": 20,
        },
        num_samples=4,
  reuse_actors=True,
loggers=[TestLogger],
        
config= {
     "lr": tune.loguniform(1e-5,1e-1)
    , "droupout_prob": tune.uniform(0,1)
          ,   "weight_decay": tune.loguniform(1e-5,1e-1),
    "b1" : tune.loguniform(1e-4, 1e-1),
    "b2" : tune.loguniform(1e-5, 1e-2)
}
                      ,resources_per_trial={'cpu':2 ,'gpu': 1})

    print("Best hyperparameters found were: ", analysis.best_config)